# Join Processed Data to School Points

In [ ]:
import pandas as pd
import geopandas as gpd

schools = gpd.read_file('../data/processed_data/school_points_with_lcgms.shp')

In [ ]:
pd.set_option('display.max_columns', None)
schools

## Join Boroughs

Lost borough names at an earlier stage so just going to bring them back in here via spatial join

In [ ]:
boroughs = gpd.read_file('../data/raw_data/NYC Planning/nybb_25c/nybb.shp')[['BoroName', 'geometry']].to_crs(schools.crs)
master_schools = gpd.sjoin(schools, boroughs, how='left', predicate='within').drop(columns=['index_right'])

## Join DACs to Schools

In [ ]:
dacs = gpd.read_file('../data/processed_data/dac_nyc_lite.geojson')

In [ ]:
# check that there aren't any public schools exactly on the border of a DAC
assert schools.geometry.apply(dacs.union_all().covers).sum() == schools.geometry.within(dacs.union_all()).sum()

In [ ]:
master_schools = gpd.sjoin(schools, dacs, how='left', predicate='within')
master_schools.drop(columns=['index_right', 'county', 'geoid'], inplace=True)
master_schools['dac_designation'] = master_schools['dac_designation'].fillna(False)

## Join Election Results to Schools

In [ ]:
primary_results = gpd.read_file('../data/processed_data/zohran_first_round_frac.geojson')

In [ ]:
# Reproject to planar CRS Web Mercator (EPSG:3857) for accurate distance calculations
master_schools_og_crs = master_schools.crs
primary_results_og_crs = primary_results.crs
master_schools = master_schools.to_crs('EPSG:3857')
primary_results = primary_results.to_crs('EPSG:3857')

# First do regular spatial join
master_schools = gpd.sjoin(master_schools, primary_results, how='left', predicate='within').drop(columns=['index_right'])

# Find unmatched schools
unmatched_mask = master_schools['ZohranFirstRoundFrac'].isna()
unmatched_schools = master_schools[unmatched_mask].copy()

print(f"Found {unmatched_mask.sum()} schools without polygon matches, using nearest neighbor...")

# Use sjoin_nearest for unmatched schools
nearest_join = gpd.tools.sjoin_nearest(unmatched_schools.drop(columns='ZohranFirstRoundFrac'), primary_results, how='left')
master_schools.loc[unmatched_mask, 'ZohranFirstRoundFrac'] = nearest_join['ZohranFirstRoundFrac'].values

assert not master_schools['ZohranFirstRoundFrac'].isna().any()

# Reproject back to original CRS
master_schools = master_schools.to_crs(master_schools_og_crs)
primary_results = primary_results.to_crs(primary_results_og_crs)

## Join A/C Data

Data scraped from NYC Schools individual webpages for each building code in LCGMS data.

In [ ]:
no_ac_summary = pd.read_csv('../data/processed_data/no_ac_summary.csv')

In [ ]:
# We are missing 185 unique building codes from our scraped A/C data
ct_bldg_codes_missing_from_ac_data = master_schools['Bldg_Code'].nunique() - no_ac_summary['BuildingCode'].nunique()

In [ ]:
master_schools = master_schools.merge(no_ac_summary, left_on='Bldg_Code', right_on='BuildingCode', how='left').drop(columns=['BuildingCode'])

In [ ]:
# Show that all the building codes we had from AC data successfully joined
assert master_schools.drop_duplicates(subset=['Bldg_Code'])['CLS_No_AC'].isna().sum() == ct_bldg_codes_missing_from_ac_data

## Join Ventilation Data

Data scraped from NYC Schools individual webpages for each building code in LCGMS data.

The `No_VT` columns here indicate that there is at least one element of *mechanical* ventilation that is non-existent. This is NOT the definition of functioning ventilation that NYC Schools website uses -- theirs allows for the existence of windows to count as adequate ventilation. However, due to the increasing frequency of air pollution from wildfire smoke and the necessity of HEPA filters in classrooms, we are instead looking at whether BOTH a supply and exhaust fan exist in a room. If a room is missing EITHER a supply fan OR an exhaust fan, it is counted in the `No_VT` column for that room type.

In [ ]:
missing_ventilation_summary = pd.read_csv('../data/processed_data/missing_ventilation_summary.csv')

In [ ]:
# We are missing 198 unique building codes from our scraped ventilation data
ct_bldg_codes_missing_from_vent_data = master_schools['Bldg_Code'].nunique() - missing_ventilation_summary['BuildingCode'].nunique()

In [ ]:
master_schools = master_schools.merge(missing_ventilation_summary, left_on='Bldg_Code', right_on='BuildingCode', how='left').drop(columns=['BuildingCode'])

In [ ]:
# Show that all the building codes we had from ventilation data successfully joined
assert master_schools.drop_duplicates(subset=['Bldg_Code'])['CLS_No_VT'].isna().sum() == ct_bldg_codes_missing_from_vent_data

## Join Building Capacity + Utilization Data

In [ ]:
bldg_capacity_utilization = pd.read_csv('../data/processed_data/bldg_capacity_utilization.csv')
bldg_capacity_utilization.rename(columns={'Bldg ID': 'Bldg_Code', 'Bldg Enroll': 'Bldg_Enroll', 'Target Bldg Cap': 'Bldg_Cap', 'Target Bldg Util': 'Bldg_Util', 'Data As Of': 'Util_As_Of'}, inplace=True)
bldg_capacity_utilization['Util_As_Of'] = pd.to_datetime(bldg_capacity_utilization['Util_As_Of'], format='%Y-%m-%d')

In [ ]:
master_schools = master_schools.merge(
    bldg_capacity_utilization[['Bldg_Code', 'Bldg_Enroll', 'Bldg_Cap', 'Bldg_Util', 'Util_As_Of']],
    on='Bldg_Code',
    how='left'
    )

## Join IBO School Barriers Data

This dataset has capacity/utilization, percentage of space with A/C, building accessibility, and some other really cool fields, and they're all already joined to location code. So hoping this will join well

NOTE: looks like IBO only included schools that were in all of the datasets they were joining (i.e. inner join for every join). This means if we go to the source data and do left joins instead, we might get better results. See footnotes on data sources [here](https://www.ibo.nyc.gov/content/publications/2025-march-barriers-to-learning-age-accessibility-space-usage-and-air-conditioning-in-nyc-school-buildings)

In [ ]:
# TODO: go back and get the original sources of all the data in IBO dataset to see if we can get better coverage.
ibo_barriers = pd.read_excel('../data/raw_data/IBO/IBO-barriers-to-learning-data-file.xlsx', sheet_name='DATA')
# NOTE: there are only 1309 records in IBO data. 
print("Pct match from IBO to master_schools:", ibo_barriers['building_code'].isin(master_schools['Bldg_Code']).sum() / len(ibo_barriers))

ibo_barriers['central_ac'] = ibo_barriers['central_ac'].map({'Y': 1, 'N': 0})

ibo_cols_of_interest = [
    'building_code',
    'building_ownership_description',
    'yearbuilt',
    'age',
    'bap_rating',
    'Accessibility_Description',
]
ibo_barriers = ibo_barriers[ibo_cols_of_interest]

In [ ]:
master_schools = master_schools.merge(ibo_barriers, left_on='Bldg_Code', right_on='building_code', how='left').drop(columns=['building_code'])

## Join Solar-Readiness Data

This DCAS dataset comes from [here](https://data.cityofnewyork.us/City-Government/City-of-New-York-Municipal-Solar-Readiness-Assessm/cfz5-6fvh/about_data) and shows which buildings with 10K+ sqft have been assessed for solar-readiness and if they are solar-ready.

In [ ]:
solar_readiness = pd.read_csv('../data/processed_data/solar_readiness_assessment_doe_buildings.csv')
# Rename columns for clarity
solar_readiness.rename(columns={
    'Site': 'Solar_Site',
    'Status': 'Solar_Status',
    'Year of Report': 'Solar_Year_of_Report',
}, inplace=True)

# Join solar readiness data
master_schools = master_schools.merge(solar_readiness, left_on='Bldg_Code', right_on='Solar_Site', how='left')#.drop(columns=['Solar_Site'])

## Join City Council Districts

In [ ]:
council_districts = gpd.read_file('../data/processed_data/city_council_districts.geojson').to_crs(master_schools.crs)
master_schools = gpd.sjoin(master_schools, council_districts, how='left', predicate='within').drop(columns=['index_right', 'BOROUGH', 'Shape_Leng', 'Shape_Area'])

# Export Joined Data

## Shorten Columns for Shapefile Limit

In [ ]:
shortened_cols = {
    # DAC columns
    'dac_designation': 'in_dac',
    'combined_score': 'comb_score',
    'percentile_rank_combined_nyc': 'pctl_comb',
    'burden_score': 'burd_score',
    'burden_score_percentile': 'pctl_burd',
    'vulnerability_score': 'vuln_score',
    'vulnerability_score_percentile': 'pctl_vuln',
    # Primary results columns
    'ZohranFirstRoundFrac': 'ZohrPrimR1',
    # IBO columns
    'age': 'Bldg_Age',
    'building_ownership_description': 'Bldg_Owner',
    'Accessibility_Description': 'Accessible',
    # Council District columns
    'NAME': 'CouncName',
    'POLITICAL PARTY': 'CouncParty',
    'DISTRICT OFFICE ADDRESS': 'CouncAddr',
    'DISTRICT OFFICE PHONE': 'CouncPhone',
    # Capacity Utilization columns
    'Bldg_Enroll': 'Bldg_Enrl',
    # Solar-Readiness columns
    'Solar_Status': 'Sol_Stat',
    'Solar_Year_of_Report': "Sol_Yr_Rep",
    'Installed or Estimated Capacity': "Sol_Capac",
    'Solar-Readiness Assessment': 'Sol_Rd_Ass',
    'Percentage of Max Peak Demand': 'SolPctDemd',
    'Estimated Annual Production': 'SolEstProd',
    'Percentage of Annual Electricity Consumption': 'SolPctCnsm',
    'Estimated Annual Emissions Reduction': 'SolEmisRed',
    'Estimated Annual Energy Savings': 'SolEnrgSav',
    'Upfront Project Cost': 'Sol_Cost',
    'Total Gross Square Footage': 'SolTotSqFt',
    'Roof Condition': 'Sol_Rf_Con',
    'Roof Age': 'Sol_Rf_Age',
    'Other Sustaibility Projects': 'Sol_Ot_Prj',
    }


# test if cols are correct length
for col in master_schools.rename(columns=shortened_cols).columns:
    if len(col) > 10:
        print(f"{col} too long: currently {len(col)} chars")

In [ ]:
# Rename columns for Shapefile
master_schools = master_schools.rename(columns=shortened_cols)

## Export to Shapefile

In [ ]:
import zipfile
import os
# Save shapefile first
shp_path = '../data/processed_data/master_schools.shp'
master_schools.sort_values('Loc_Code').to_file(
    shp_path,
    driver='ESRI Shapefile'
)

# Create zip file with all shapefile components
zip_path = '../data/processed_data/master_schools.zip'
base_name = '../data/processed_data/master_schools'

# Shapefile extensions to include
extensions = ['.shp', '.shx', '.dbf', '.prj', '.cpg']

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for ext in extensions:
        file_path = base_name + ext
        if os.path.exists(file_path):
            # Add file to zip with just the filename (no path)
            zipf.write(file_path, os.path.basename(file_path))
            print(f"Added {os.path.basename(file_path)} to zip")

print(f"Shapefile saved as zip: {zip_path}")

## Export to GeoJSON

In [ ]:
master_schools.sort_values('Loc_Code').to_file(
    '../data/processed_data/master_schools.geojson', driver='GeoJSON'
)